<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=f54719ffce7621e7782c026da0733dfca5ef96f102357da847895216c240b105
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7


Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-03 11:11:09
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.40 C
-------------------
Today PnL: 3.29 K (0.02%)
Current PnL: -21.23 L (-14.09%)
CY Booked + Current PnL: -9.79 L (-6.5%)
-------------------
Total profit:  1.29 L
Total loss:  -22.52 L
-------------------
Total Booked + Current PnL: 17.19 L (13.9%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.15%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.54 L (63.08%)
Deployed:  1.24 C
Current:  1.40 C
CAGR/XIRR %: 7.71%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-41.18,69.0,H-MC,13.02,117490.0,16986.0,12125.0,3.35,16.90,10.32,28.96,79.0,1.40,0.84,56.61,MH,ATH,METALS
77,TTKPRESTIG,770.00,94.66,40.0,M-SC,1.70,83840.0,-16937.0,17028.0,0.46,-16.81,20.31,0.09,245.0,-0.99,0.60,10.15,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,8.25,43.0,H-MC,2.90,186850.0,22180.0,21151.0,-0.50,13.47,11.32,26.31,89.0,1.05,1.34,22.53,X40N,NTT,AC
42,ITC,452.00,-38.77,46.0,H-LC,2.17,197800.0,-2338.0,23222.0,-0.27,-1.17,11.74,10.44,4.0,-0.10,1.42,4.33,X40,NTT,FMCG
34,ICICIGI,2260.25,-17.79,55.0,H-MC,1.75,141244.0,5111.0,26017.0,0.27,3.75,18.42,22.86,91.0,0.20,1.01,16.84,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,TANLA,1963.11,-17.86,58.0,H-SC,18.56,220427.0,-50334.0,380281.0,4.34,-18.59,172.52,121.86,133.0,-0.13,1.58,72.44,AR,ATH,IT
50,NATIONALUM,244.55,-41.18,69.0,H-MC,13.02,117490.0,16986.0,12125.0,3.35,16.90,10.32,28.96,79.0,1.40,0.84,56.61,MH,ATH,METALS
24,DEN,75.00,99.48,47.0,M-SC,1.94,55888.0,-28576.0,64115.0,2.74,-33.83,114.72,42.07,237.0,-0.45,0.40,18.69,AR,NTT,ENTERTAINMENT
32,HINDZINC,730.22,31.07,68.0,M-LC,4.48,214150.0,9074.0,103499.0,2.27,4.42,48.33,54.89,52.0,0.09,1.53,29.59,X5K,ATH,METALS
12,BATAINDIA,2096.00,-32.72,54.0,M-SC,4.77,100613.0,-28057.0,73357.0,2.03,-21.81,72.91,35.20,219.0,-0.38,0.72,15.48,X40,NTT,FOOTWEAR


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-184.09,70.0,M-SC,9.35,145359.0,-3861.0,148048.0,-4.76,-2.59,101.85,96.62,208.0,-0.03,1.04,57.31,XY25,NTT,FINANCE
64,SHALBY,327.00,1214.19,64.0,M-SC,18.12,178690.0,-2453.0,47925.0,-2.39,-1.35,26.82,25.10,235.0,-0.05,1.28,43.20,XY24,NTT,HEALTHCARE
66,SIS,528.00,2069.43,39.0,H-SC,3.70,87554.0,-23478.0,47087.0,-1.65,-21.15,53.78,21.26,156.0,-0.50,0.63,18.29,OX40N,NTT,MISC
49,MASFIN,398.61,-20.05,40.0,H-SC,11.55,90645.0,-7335.0,28934.0,-1.53,-7.49,31.92,22.05,152.0,-0.25,0.65,31.76,XR,ATH,FINANCE
20,COALINDIA,497.97,5.97,42.0,L-LC,4.23,141747.0,-8714.0,42496.0,-1.39,-5.79,29.98,22.46,167.0,-0.21,1.01,11.24,XY25,ATH,MINING


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.0,-21.57,46.0,H-MC,2.19,135035.0,-753.0,43508.0,0.15,-0.55,32.22,31.48,107.0,-0.02,0.97,11.5,X40,ATH,INSURANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,105.37,64.0,H-SC,12.07,129654.0,-12825.0,32349.0,-0.44,-9.00,24.95,13.70,163.0,-0.40,0.93,53.03,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.24,60.0,H-SC,1.85,226458.0,-43209.0,79532.0,0.96,-16.02,35.12,13.47,138.0,-0.54,1.62,14.54,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,94.66,40.0,M-SC,1.70,83840.0,-16937.0,17028.0,0.46,-16.81,20.31,0.09,245.0,-0.99,0.60,10.15,OX40N,NTT,DURABLES
18,CERA,9475.0,-22.43,40.0,H-SC,1.47,141611.0,-34292.0,76314.0,0.84,-19.49,53.89,23.89,149.0,-0.45,1.01,22.30,OX40N,NTT,CERAMICS
66,SIS,528.0,2069.43,39.0,H-SC,3.70,87554.0,-23478.0,47087.0,-1.65,-21.15,53.78,21.26,156.0,-0.50,0.63,18.29,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,49.16,63.0,M-MC,11.98,236017.0,11055.0,156597.0,1.59,4.91,66.35,74.53,192.0,0.07,1.69,38.75,XY24,BTT,STEEL
32,HINDZINC,730.22,31.07,68.0,M-LC,4.48,214150.0,9074.0,103499.0,2.27,4.42,48.33,54.89,52.0,0.09,1.53,29.59,X5K,ATH,METALS
14,BLUESTARCO,2080.00,8.25,43.0,H-MC,2.90,186850.0,22180.0,21151.0,-0.50,13.47,11.32,26.31,89.0,1.05,1.34,22.53,X40N,NTT,AC
84,VOLTAS,1530.00,-3.60,43.0,H-MC,2.19,202320.0,10578.0,27172.0,-0.35,5.52,13.43,19.69,99.0,0.39,1.45,12.69,XY25,NTT,AC
17,CAMS,5211.76,-6.05,45.0,H-SC,1.98,106904.0,4900.0,39020.0,-0.14,4.80,36.50,43.06,122.0,0.13,0.76,22.74,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-41.18,69.0,H-MC,13.02,117490.0,16986.0,12125.0,3.35,16.90,10.32,28.96,79.0,1.40,0.84,56.61,MH,ATH,METALS
11,BANKINDIA,190.00,-26.71,64.0,H-MC,9.99,190404.0,10596.0,101809.0,-0.67,5.89,53.47,62.52,88.0,0.10,1.36,41.99,XR,NTT,BANKS
32,HINDZINC,730.22,31.07,68.0,M-LC,4.48,214150.0,9074.0,103499.0,2.27,4.42,48.33,54.89,52.0,0.09,1.53,29.59,X5K,ATH,METALS
37,INDIAMART,4850.92,-54.45,37.0,H-SC,9.40,126935.0,3599.0,130159.0,-0.16,2.92,102.54,108.45,119.0,0.03,0.91,25.92,AR,ATH,MISC
86,WIPRO,420.00,-15.15,40.0,M-LC,5.87,148726.0,-2219.0,111678.0,-0.49,-1.47,75.09,72.51,53.0,-0.02,1.06,4.46,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.42,29.0,H-LC,5.80,205911.0,-45857.0,99064.0,0.18,-18.21,48.11,21.13,27.0,-0.46,1.47,10.76,X40,ATH,PAINTS
83,VBL,671.64,-22.15,30.0,H-LC,9.58,280210.0,-35632.0,148287.0,-0.85,-11.28,52.92,35.67,5.0,-0.24,2.00,1.01,X40N,ATH,FMCG
74,TCS,4476.75,-31.58,31.0,H-LC,12.47,272177.0,-73779.0,148636.0,-0.64,-21.33,54.61,21.64,1.0,-0.50,1.95,0.25,X40,ATH,IT
30,HCLTECH,1943.91,-4.95,33.0,H-LC,10.31,217508.0,-24416.0,87677.0,-0.29,-10.09,40.31,26.15,8.0,-0.28,1.56,2.78,X40,ATH,IT
40,INFY,2275.00,-21.03,38.0,H-LC,8.24,306997.0,-5891.0,177567.0,-0.31,-1.88,57.84,54.87,3.0,-0.03,2.20,4.64,X40,BTT,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,-2.61,44.0,H-LC,1.74,156395.0,-29700.0,77181.0,0.21,-15.96,49.35,25.51,15.0,-0.38,1.12,15.25,AR,ATH,ELECTRICAL
48,LTIM,7230.2,-6.94,43.0,H-LC,1.80,230535.0,-15616.0,94819.0,0.00,-6.34,41.13,32.18,16.0,-0.16,1.65,27.41,X200,ATH,IT
42,ITC,452.0,-38.77,46.0,H-LC,2.17,197800.0,-2338.0,23222.0,-0.27,-1.17,11.74,10.44,4.0,-0.10,1.42,4.33,X40,NTT,FMCG
57,RELIANCE,1533.0,-15.17,38.0,H-LC,3.54,212035.0,1669.0,27119.0,-0.69,0.79,12.79,13.68,37.0,0.06,1.52,17.45,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-11.91,43.0,H-LC,3.76,271253.0,5827.0,50968.0,-0.62,2.20,18.79,21.40,11.0,0.11,1.94,9.44,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,31.0,H-LC,12.47,272177.0,-73779.0,148636.0,-0.64,-21.33,54.61,21.64,1.0,-0.50,1.95,0.25,X40,ATH,IT
83,VBL,671.64,-22.15,30.0,H-LC,9.58,280210.0,-35632.0,148287.0,-0.85,-11.28,52.92,35.67,5.0,-0.24,2.00,1.01,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.95,33.0,H-LC,10.31,217508.0,-24416.0,87677.0,-0.29,-10.09,40.31,26.15,8.0,-0.28,1.56,2.78,X40,ATH,IT
42,ITC,452.00,-38.77,46.0,H-LC,2.17,197800.0,-2338.0,23222.0,-0.27,-1.17,11.74,10.44,4.0,-0.10,1.42,4.33,X40,NTT,FMCG
40,INFY,2275.00,-21.03,38.0,H-LC,8.24,306997.0,-5891.0,177567.0,-0.31,-1.88,57.84,54.87,3.0,-0.03,2.20,4.64,X40,BTT,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-2.61,44.0,H-LC,1.74,156395.0,-29700.0,77181.0,0.21,-15.96,49.35,25.51,15.0,-0.38,1.12,15.25,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,46.0,H-LC,2.17,197800.0,-2338.0,23222.0,-0.27,-1.17,11.74,10.44,4.0,-0.10,1.42,4.33,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-18.42,29.0,H-LC,5.80,205911.0,-45857.0,99064.0,0.18,-18.21,48.11,21.13,27.0,-0.46,1.47,10.76,X40,ATH,PAINTS
19,CIPLA,1795.00,-22.52,42.0,H-LC,3.90,207158.0,2658.0,38759.0,-0.06,1.30,18.71,20.25,10.0,0.07,1.48,10.15,X40N,BTT,PHARMA
57,RELIANCE,1533.00,-15.17,38.0,H-LC,3.54,212035.0,1669.0,27119.0,-0.69,0.79,12.79,13.68,37.0,0.06,1.52,17.45,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,31.0,H-LC,12.47,272177.0,-73779.0,148636.0,-0.64,-21.33,54.61,21.64,1.0,-0.50,1.95,0.25,X40,ATH,IT
40,INFY,2275.00,-21.03,38.0,H-LC,8.24,306997.0,-5891.0,177567.0,-0.31,-1.88,57.84,54.87,3.0,-0.03,2.20,4.64,X40,BTT,IT
42,ITC,452.00,-38.77,46.0,H-LC,2.17,197800.0,-2338.0,23222.0,-0.27,-1.17,11.74,10.44,4.0,-0.10,1.42,4.33,X40,NTT,FMCG
83,VBL,671.64,-22.15,30.0,H-LC,9.58,280210.0,-35632.0,148287.0,-0.85,-11.28,52.92,35.67,5.0,-0.24,2.00,1.01,X40N,ATH,FMCG
1,ABB,7934.00,-40.55,44.0,H-LC,7.66,248328.0,-13291.0,132508.0,-0.49,-5.08,53.36,45.57,7.0,-0.10,1.78,5.67,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,47.0,L-SC,37.56,80640.0,-32909.0,72963.0,1.40,-28.98,90.48,35.27,268.0,-0.45,0.58,97.37,XR,NTT,HOTELS
49,MASFIN,398.61,-20.05,40.0,H-SC,11.55,90645.0,-7335.0,28934.0,-1.53,-7.49,31.92,22.05,152.0,-0.25,0.65,31.76,XR,ATH,FINANCE
50,NATIONALUM,244.55,-41.18,69.0,H-MC,13.02,117490.0,16986.0,12125.0,3.35,16.90,10.32,28.96,79.0,1.40,0.84,56.61,MH,ATH,METALS
78,UJJIVANSFB,60.00,105.37,64.0,H-SC,12.07,129654.0,-12825.0,32349.0,-0.44,-9.00,24.95,13.70,163.0,-0.40,0.93,53.03,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,70.0,M-SC,9.35,145359.0,-3861.0,148048.0,-4.76,-2.59,101.85,96.62,208.0,-0.03,1.04,57.31,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1214.19,64.0,M-SC,18.12,178690.0,-2453.0,47925.0,-2.39,-1.35,26.82,25.10,235.0,-0.05,1.28,43.20,XY24,NTT,HEALTHCARE
69,SURYODAY,240.00,73.25,81.0,H-SC,3.91,159844.0,-19227.0,84957.0,0.72,-10.74,53.15,36.71,135.0,-0.23,1.14,58.60,XR,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,70.0,M-SC,9.35,145359.0,-3861.0,148048.0,-4.76,-2.59,101.85,96.62,208.0,-0.03,1.04,57.31,XY25,NTT,FINANCE
71,TANLA,1963.11,-17.86,58.0,H-SC,18.56,220427.0,-50334.0,380281.0,4.34,-18.59,172.52,121.86,133.0,-0.13,1.58,72.44,AR,ATH,IT
7,ATULAUTO,844.00,3586.49,50.0,M-SC,7.85,165529.0,-31603.0,121432.0,-0.68,-16.03,73.36,45.57,236.0,-0.26,1.18,17.36,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.97
1,25,43.14
2,50,74.79


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.27
LC    32.01
MC    23.72
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.92
X40      14.45
X40N     11.22
XR       11.10
XY25     10.35
AR        8.18
OX40N     7.79
MH        1.65
X200      1.65
X5K       1.53
SR        1.16
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.12
H-LC    25.41
H-MC    20.41
M-SC    13.70
M-LC     5.59
M-MC     3.01
L-SC     1.45
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.76,-6.71,41.93
IT,12.52,-20.15,85.67
FINANCE,10.39,-16.18,68.95
MISC,6.93,-18.67,84.64
BANKS,6.42,-11.11,69.56
PAINTS,5.59,-18.17,35.78
ELECTRICAL,5.33,-10.62,49.44
INSURANCE,3.75,-5.50,44.93
AUTO,3.57,-14.95,69.90


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3250197.0,22
XR,1356956.0,14
AR,1188944.0,9
X40,831139.0,10
X40N,636523.0,9
OX40N,558058.0,10
XY25,491329.0,7
SR,259255.0,2
X5K,103499.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3722638.0,29
M-SC,1514206.0,17
H-LC,1270246.0,15
H-MC,1253045.0,15
M-LC,368221.0,4
M-MC,363369.0,2
L-SC,259736.0,3
L-MC,60167.0,1
L-LC,42496.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1230726.0      6
M-SC       XY24       856057.0      7
H-SC       AR         812054.0      5
           XR         782136.0      7
H-MC       XY24       572704.0      4
H-LC       X40        536166.0      5
M-MC       XY24       363369.0      2
H-SC       X40N       311701.0      4
           SR         259255.0      2
           OX40N      255486.0      4
H-LC       X40N       237713.0      3
H-MC       X40        221616.0      4
H-LC       AR         209689.0      2
H-MC       XY25       183223.0      2
L-SC       XR         170779.0      2
M-SC       AR         167201.0      2
M-LC       XY24       153044.0      2
M-SC       XY25       148048.0      1
           OX40N      139156.0      4
           XR         130387.0      2
H-LC       XY25       117562.0      3
M-LC       XR         111678.0      1
           X5K        103499.0      1
H-MC       XR         101809.0      1
H-LC       X200        94819.0      1
L-SC       OX40N       88957.0      1
H-MC       X40N        87109.0      2
           OX40N       74459.0      1
H-LC       XY24        74297.0      1
M-SC       X40         73357.0      1
H-SC       MH          71280.0      1
L-MC       XR          60167.0      1
L-LC       XY25        42496.0      1
H-MC       MH          12125.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
